In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
dataset = pd.read_csv("/kaggle/input/d/omarnasr/ai-image-classification-2/image_labels.tsv",sep="\t")

In [2]:
!git clone https://github.com/Omar-Nasr/ai_generated_image_classification_defactify_2024

Cloning into 'ai_generated_image_classification_defactify_2024'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 130 (delta 76), reused 80 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 68.91 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [3]:
import sys
sys.path.append("/kaggle/working/ai_generated_image_classification_defactify_2024")

In [4]:
!cd /kaggle/working/ai_generated_image_classification_defactify_2024 && git pull

Already up to date.


In [5]:
from data_handler import Image_Classification_Dataset

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [6]:
!mkdir checkpoints

In [7]:
train_data_dir = "/kaggle/input/d/omarnasr/ai-image-classification-2/" 
checkpoint_path = "checkpoints/"
val_data_dir = "/kaggle/input/ai-image-classification-val-test/"

In [8]:
!curl "https://drive.usercontent.google.com/download?id=1mutiXZZGP9UhtqOLSDspJHX7VNafT2z0&export=download&authuser=0&confirm=t&uuid=85b29666-6ba1-4a29-830c-7bc011d7a176&at=AENtkXb0JST3mpXXQql8s83jTM4M:1730885596175" --output val.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  226k  100  226k    0     0   122k      0  0:00:01  0:00:01 --:--:--  122k


In [9]:
!unzip -P DEFACTIFY-4_IMG_VAL "/kaggle/working/val.zip"

Archive:  /kaggle/working/val.zip
   creating: val_dataset/
  inflating: val_dataset/Readme_Val_data.md  
  inflating: val_dataset/Validation_labels.xlsx  


In [10]:
val_labels_df = pd.read_excel("/kaggle/working/val_dataset/Validation_labels.xlsx")
val_labels_df

,Index,Caption,Label_A,Label_B
0,0,a toilet sits next to a shower an sink,1,3
1,1,A TV sitting on top of a table next to a lapto...,0,0
2,2,Two giraffes eat from a pot attached to a fence.,1,1
3,3,The kitchen has many grill with pots hanging a...,1,4
4,4,some road signs besides a road in the street,1,1
...,...,...,...,...
8995,8995,Three red double decker buses parked side by s...,0,0
8996,8996,A white toilet in a white tiled bathroom.,1,3
8997,8997,A bus is stopping to pick up a passenger.,0,0
8998,8998,Small birds in the grass in the sun,1,4


In [11]:

val_labels = np.array(val_labels_df.iloc[:,2])
val_multiclass_labels = np.array(val_labels_df.iloc[:,3])
val_multiclass_labels

array([3, 0, 1, ..., 0, 4, 2])

In [12]:
from data_handler import Image_Classification_Dataset
from torch.utils.data import DataLoader
val = Image_Classification_Dataset(val_data_dir,task="Binary",val=True,val_labels=val_labels)
test = Image_Classification_Dataset(val_data_dir,task="Binary",val=False,test=True)
test_loader = DataLoader(test,16,num_workers=4)

In [13]:
from models import train_classifier
model_adopt_binary,classifier_adopt = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_labels,val=True,use_fourier=False,optimizer_name="adopt")

Downloading: "https://download.pytorch.org/models/swin_v2_b-781e5279.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_b-781e5279.pth
100%|██████████| 336M/336M [00:05<00:00, 68.3MB/s]


Epoch 0/2

----------



libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'gXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmnd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmdd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'vued': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'view': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'lumi

Epoch $0 Training Loss: $714726.4584867003 

Epoch $0 Training F1_Score 0.8651382327079773 Best F1: 0.8651382327079773


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


...

Epoch $1 Val Loss: $20967.03169532685 

Epoch $1 Val F1_Score 0.9225614070892334 Best F1: 0.9225614070892334
Epoch 2/2

----------



libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'gXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmnd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmdd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'vued': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'view': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'lumi

Epoch $2 Training Loss: $66976.35148278598 

Epoch $2 Training F1_Score 0.9203969240188599 Best F1: 0.9225614070892334


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch $2 Val Loss: $9684.574603267269 

Epoch $2 Val F1_Score 0.9406049847602844 Best F1: 0.9406049847602844
Training complete in 62m 16s

Best F1: 0.9406049847602844 after 3 Epochs


In [14]:
from models import train_classifier
model_adopt_multiclass,classifier_adopt_multiclass = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",use_fourier=False,optimizer_name="adopt")

Epoch 0/2

----------



libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'gXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmnd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmdd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'vued': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'view': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'lumi

Epoch $0 Training Loss: $1770859458.6356564 

Epoch $0 Training F1_Score 0.19669046998023987 Best F1: 0.19669046998023987


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch $0 Val Loss: $164340153.2555542 

Epoch $0 Val F1_Score 0.14722222089767456 Best F1: 0.14722222089767456
Epoch 1/2

----------



...

Epoch $1 Val Loss: $62332200.46127319 

Epoch $1 Val F1_Score 0.16333332657814026 Best F1: 0.16333332657814026
Epoch 2/2

----------



libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'gXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmnd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmdd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'vued': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'view': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'lumi

Epoch $2 Training Loss: $95218369.50240326 

Epoch $2 Training F1_Score 0.2274523824453354 Best F1: 0.2274523824453354


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch $2 Val Loss: $35584941.77056885 

Epoch $2 Val F1_Score 0.14677777886390686 Best F1: 0.16333332657814026
Training complete in 62m 23s

Best F1: 0.2274523824453354 after 3 Epochs


In [15]:
from models import train_classifier
model_adam_binary,classifier_adam = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_labels,val=True,use_fourier=False)
model_adam_multiclass,classifier_adam_multiclass = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",use_fourier=False)

Epoch 0/2

----------



libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'gXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmnd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmdd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'vued': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'view': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'lumi

Epoch $0 Training Loss: $4495.9720149125205 

Epoch $0 Training F1_Score 0.9788609147071838 Best F1: 0.9788609147071838


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch $0 Val Loss: $708.4806743563386 

Epoch $0 Val F1_Score 0.9866209626197815 Best F1: 0.9866209626197815
Epoch 1/2

----------



...

Epoch $1 Val Loss: $47081.3376955986 

Epoch $1 Val F1_Score 0.5108888745307922 Best F1: 0.5108888745307922
Epoch 2/2

----------



libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'gXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmnd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmdd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'vued': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'view': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'lumi

Epoch $2 Training Loss: $4370.505247467605 

Epoch $2 Training F1_Score 0.9674047827720642 Best F1: 0.9674047827720642


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch $2 Val Loss: $59272.97860908508 

Epoch $2 Val F1_Score 0.49977776408195496 Best F1: 0.5108888745307922
Training complete in 62m 22s

Best F1: 0.9674047827720642 after 3 Epochs


In [16]:
from torchvision import models
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_adopt_binary.to(device)
classifier_adopt.to(device)
model_adopt_multiclass.to(device)
classifier_adopt_multiclass.to(device)
model_adam_multiclass.to(device)
classifier_adam_multiclass.to(device)
test_preds_adam_binary = []
test_preds_adam_multiclass = []
test_preds_adopt_binary = []
test_preds_adopt_multiclass = []
for inputs in test_loader:
        inputs = inputs.to(device)
        features = model_adam_multiclass(inputs)
        outputs = classifier_adam_multiclass(features)
        _,preds = torch.max(outputs,1)
        test_preds_adam_multiclass.append(preds.cpu())
for inputs in test_loader:
        inputs = inputs.to(device)
        features = model_adam_binary(inputs)
        outputs = classifier_adam(features)
        _,preds = torch.max(outputs,1)
        test_preds_adam_binary.append(preds.cpu())    
for inputs in test_loader:
        inputs = inputs.to(device)
        features = model_adopt_multiclass(inputs)
        outputs = classifier_adopt_multiclass(features)
        _,preds = torch.max(outputs,1)
        test_preds_adopt_multiclass.append(preds.cpu())
for inputs in test_loader:
        inputs = inputs.to(device)
        features = model_adopt_binary(inputs)
        outputs = classifier_adopt(features)
        _,preds = torch.max(outputs,1)
        test_preds_adopt_binary.append(preds.cpu())    
test_preds_adopt_binary = np.concatenate(test_preds_adopt_binary)
test_preds_adopt_multiclass = np.concatenate(test_preds_adopt_multiclass)
test_preds_adam_binary = np.concatenate(test_preds_adam_binary)
test_preds_adam_multiclass = np.concatenate(test_preds_adam_multiclass)


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB pro

In [17]:
from datasets import load_dataset
import json
ds = load_dataset("NasrinImp/Defactify4_Test")
captions = ds["train"]['caption']
list = []
for i,pred in enumerate(test_preds_adam_binary):
    dict = {}
    dict["index"]=i
    dict["caption"]=captions[i]
    dict["Label_A"]=int(test_preds_adam_binary[i])
    dict["Label_B"]=int(test_preds_adam_multiclass[i])
    list.append(dict)
json_object = json.dumps(list,indent=4)
with open("answer_adam.json","w") as f:
    f.write(json_object)

README.md:   0%|          | 0.00/320 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/431M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

In [18]:
from datasets import load_dataset
import json
ds = load_dataset("NasrinImp/Defactify4_Test")
captions = ds["train"]['caption']
list = []
for i,pred in enumerate(test_preds_adopt_binary):
    dict = {}
    dict["index"]=i
    dict["caption"]=captions[i]
    dict["Label_A"]=int(test_preds_adopt_binary[i])
    dict["Label_B"]=int(test_preds_adopt_multiclass[i])
    list.append(dict)
json_object = json.dumps(list,indent=4)
with open("answer_adopt.json","w") as f:
    f.write(json_object)